In [1]:
from tensorflow.keras.models import load_model
from glob import glob
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from pyeit.mesh.wrapper import PyEITAnomaly_Circle
from src.vae2D import vae_model
from src.util import (
    seq_data, 
    load_2Ddata, 
    seq_data, 
    compute_perm_deviation, 
    calculate_perm_error, 
    select_random_instances,
    plot_boxplot,
    mesh_plot_comparisons,
    plot_mesh_permarray,
    load_exp_data,
    plot_mesh
)
from pyeit import mesh
from keras import backend as K
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras.models import Model
from pyeit import mesh
from tqdm import tqdm
from datetime import datetime
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

2025-03-08 11:39:13.364165: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741433953.379225 4079027 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741433953.383410 4079027 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 11:39:13.398474: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
W0000 00:00:1741433963.394214 4079027 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libr

## load training data (trajectory that should be used for training)

In [ ]:
voltage, gamma, _ = load_2Ddata("exp_data", "exp_data_set", data_type = "exp") 

## loading VAE model and generating labels for supervised learning 

In [6]:
vae = vae_model()  
vae.build(input_shape=(None, 2840)) 
vae.load_weights("weights/vae2D.weights.h5")
print("Weights of VAE successfully loaded.")

Gewichte erfolgreich geladen.


In [7]:
_, _, latent_rep = vae.encoder.predict(gamma)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step


## mapper (with or without LSTM) training

In [11]:
(
    v_train_seq,
    v_test_seq,
    gamma_train,
    gamma_test,
    z_train,
    z_test,
) = train_test_split(
    voltage,
    gamma,
    latent_rep,
    test_size=0.1,
    shuffle=True,
    random_state=42,
)

In [12]:
from src.lstm_mapper2D import lstm_mapper2D_model
lstm_mapper = mapper_model()
lstm_mapper.summary()
lstm_mapper.compile(optimizer="adam", loss="mse")

Model: "lstm_mapper"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 4, 32, 32, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 4, 15, 15, 2)   │            20 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 4, 7, 7, 2)     │            38 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 4, 98)          │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 4, 64)          │         6,336 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 4, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 16)             │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           136 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,082 (86.26 KB)

 Trainable params: 22,082 (86.26 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from src.mapper2D import no_lstm_mapper2D_model
mapper = no_lstm_mapper_model()
mapper.summary()
mapper.compile(optimizer="adam", loss="mse")

In [ ]:
#stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
#history_mapper = lstm_mapper.fit(v_train_seq, z_train, epochs=100, batch_size = 8, validation_split = 0.1, callbacks=[stop_early])
history_mapper = lstm_mapper.fit(v_train_seq, z_train, epochs=100, batch_size = 8, validation_split = 0.1)
#history_mapper = mapper.fit(v_train_seq, z_train, epochs=100, batch_size=8, validation_split=0.1)

Epoch 1/100
6073/6073 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - loss: 0.4063 - val_loss: 0.1744
Epoch 2/100
6073/6073 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 0.1675 - val_loss: 0.1551
Epoch 3/100
6073/6073 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 0.1556 - val_loss: 0.1524
Epoch 4/100
6073/6073 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 0.1514 - val_loss: 0.1524
Epoch 5/100
6073/6073 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 0.1480 - val_loss: 0.1503
Epoch 6/100
6073/6073 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 0.1462 - val_loss: 0.1484
Epoch 7/100
6073/6073 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 0.1448 - val_loss: 0.1457
Epoch 8/100
6073/6073 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 0.1455 - val_loss: 0.1456
Epoch 9/100
6073/6073 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 0.1430 - val_loss: 0.1470
Epoch 10/100
6073/6073 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 0.1440 - val_loss: 0.1450
Epoch 11/100
6073/6073 ━━━━━━━━━━━━━━━━━━━━ 30s 5ms/step - loss: 0.1424 - val_loss: 0.1455
Epoch 12

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(history_mapper.history["loss"], label="Training Loss")
plt.plot(history_mapper.history["val_loss"], label="Validation Loss")
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Loss", fontsize=15)
plt.legend()
plt.tight_layout()
plt.grid(True, linestyle=':', color='gray', alpha=0.5)
plt.show()

## saving weights of trained model

In [ ]:
weights_dir = "weights/"
os.makedirs(weights_dir, exist_ok=True)
weights_file = os.path.join(weights_dir, "lstm_exp_2D.weights.h5")
lstm_mapper.save_weights(weights_file)
#mapper.save_weights(weights_file)
print(f"weights stored under: {weights_file}")